In [23]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import os
%matplotlib qt

### Camera calibration

In [24]:
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

objpoints = [] 
imgpoints = [] 
images = glob.glob('camera_cal/calibration*.jpg')
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
cv2.destroyAllWindows()
img = cv2.imread('camera_cal/calibration18.jpg')
img_size = (img.shape[1], img.shape[0])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
dst = cv2.undistort(img, mtx, dist, None, mtx)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "camera_cal/dist_pickle.p", "wb" ) )
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)
f.savefig('camera_cal/calibrated')

### Loading camera calib

In [25]:
dist_pickle = pickle.load( open( "camera_cal/dist_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]
#img = cv2.imread('test_images/straight_lines1.jpg')

#dst = cv2.undistort(img,mtx,dist,None,mtx)
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
#ax1.imshow(img)
#ax1.set_title('Original Image', fontsize=30)
#ax2.imshow(dst)
#ax2.set_title('Undistorted Image', fontsize=30)


### Helpers

In [26]:
def hls_select(img, thresh=(0, 255),channel = 2):
    hls = cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    if channel == 0 :
        chan = H
    if channel == 1 :
        chan = L 
    if channel == 2 :
        chan = S 
    binary = np.zeros_like(chan)
    binary[(chan>thresh[0])&(chan<=thresh[1])] = 1
    return binary

def rgb_select(img, thresh=(0, 255),channel = 0):
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    if channel == 0 :
        chan = R
    if channel == 1 :
        chan = G 
    if channel == 2 :
        chan = B 
    binary = np.zeros_like(chan)
    binary[(chan>thresh[0])&(chan<=thresh[1])] = 1
    return binary

def convert_to_grayscale(img):
    return cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

def abs_sobel_thresh(img, orient='x',sobel_kernel = 3,thresh=(0,255)):
    gray = convert_to_grayscale(img)
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize = sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize = sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = convert_to_grayscale(img)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = convert_to_grayscale(img)
    sobelx = cv2.Sobel(gray,cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    abs_sobel_x = np.absolute(sobelx)
    abs_sobel_y = np.absolute(sobely)
    abs_dir = np.arctan2(abs_sobel_y,abs_sobel_x)
    binary_output =  np.zeros_like(abs_dir)
    binary_output[(abs_dir >= thresh[0]) & (abs_dir <= thresh[1])] = 1
    return binary_output

def undistort(img):
    return cv2.undistort(img,mtx,dist,None,mtx)




def hist(img):
    bottom_half = img[img.shape[0]//2:,:]
    histogram = np.sum(bottom_half, axis=0)
    return histogram

### Compare different color channels

In [27]:
import os
for name in os.listdir("imagini/"):
    image = mpimg.imread('imagini/'+ name)
    dst = undistort(image)
    binary_h = hls_select(dst,(15,100),channel = 0)
    binary_l = hls_select(dst,(130,190),channel = 1)
    binary_s = hls_select(dst,(40,215),channel = 2)
    binary_r = rgb_select(dst,(120,255),channel = 0)
    binary_g = rgb_select(dst,(135,255),channel = 1)
    binary_b = rgb_select(dst,(195,255),channel = 2)
    combined_ls = np.zeros_like(binary_l)
    combined_ls[(binary_l == 1) & (binary_s == 1)] = 1
    
    combined_rb = np.zeros_like(binary_r)
    combined_rb[(binary_r == 1) & (binary_b == 1)] = 1
    
    # best combination
    combined_ls_rb = np.zeros_like(binary_r)
    combined_ls_rb[(combined_ls == 1) | (combined_rb == 1)] = 1
    
    f, ((ax1, ax2,ax3),(ax4, ax5,ax6),(ax7, ax8,ax9)) = plt.subplots(3, 3, figsize=(17,14))
    f.tight_layout()
    ax1.imshow(binary_h,cmap='gray')
    ax2.imshow(binary_l,cmap='gray')
    ax3.imshow(binary_s,cmap='gray')
    ax4.imshow(binary_r,cmap='gray')
    ax5.imshow(binary_g,cmap='gray')
    ax6.imshow(binary_b,cmap='gray')
    ax7.imshow(combined_ls,cmap='gray')
    ax8.imshow(combined_rb,cmap='gray')
    ax9.imshow(combined_ls_rb,cmap='gray')
    

### Compare different gradients

In [28]:
for name in os.listdir("test_images/"):
    image = mpimg.imread('test_images/'+ name)
    dst = undistort(image)
    ksize = 3 
    image = dst
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(20, 150))
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(20, 100))
    mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(30, 100))
    dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0.7, 1.3))
    combined_grad = np.zeros_like(gradx)
    combined_grad[((gradx == 1) & (grady == 1))] = 1
    combined_md = np.zeros_like(dir_binary)
    combined_md[((mag_binary == 1) & (dir_binary == 1))] = 1
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    combined_gradx_mag = np.zeros_like(gradx)
    combined_gradx_mag[((gradx == 1) & (mag_binary == 1))] = 1
    combined_grady_mag = np.zeros_like(grady)
    combined_grady_mag[((grady == 1) & (mag_binary == 1))] = 1
    combined_gradx_dir = np.zeros_like(gradx)
    combined_gradx_dir[((gradx == 1) & (dir_binary == 1))] = 1
    combined_grady_dir = np.zeros_like(grady)
    combined_grady_dir[((grady == 1) & (dir_binary == 1))] = 1
    plt.imshow(grady,cmap='gray')
    
    
    f, ((ax1, ax2,ax3,ax4),(ax5,ax6,ax7,ax8),(ax9,ax10,ax11,ax12)) = plt.subplots(3, 4, figsize=(20,20))
    f.tight_layout()
    ax1.set_title('GRADX')
    ax1.imshow(gradx,cmap='gray')
    ax2.set_title('GRADY')
    ax2.imshow(grady,cmap='gray')
    ax3.set_title('Combined GRAD')
    ax3.imshow(combined_grad,cmap='gray') 
    ax4.set_title('Magnitude')
    ax4.imshow(mag_binary, cmap='gray')
    ax5.set_title('Direction')
    ax5.imshow(dir_binary, cmap='gray')
    ax6.set_title('Combined mag-dir')
    ax6.imshow(combined_md,cmap='gray') 
    ax7.set_title('Combined gradx-mag')
    ax7.imshow(combined_gradx_mag,cmap='gray')
    ax8.set_title('Combined grady-mag')
    ax8.imshow(combined_grady_mag,cmap='gray')
    ax9.set_title('Combined gradx-dir')
    ax9.imshow(combined_gradx_mag,cmap='gray')
    ax10.set_title('Combined grady-dir')
    ax10.imshow(combined_grady_mag,cmap='gray')
    ax11.set_title('Combined all')
    ax11.imshow(combined,cmap='gray')
    

### WARP

In [29]:

def warp(img):
    img_size = (img.shape[1],img.shape[0])
    src = np.float32([[820,510],[1200,690],[520,510],[320,690]])
    dst = np.float32([[950,0],[950,719],[290,0],[290,719]])
    M = cv2.getPerspectiveTransform(src,dst)
    warped = cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
    return warped

def unwarp(img):
    img_size = (img.shape[1],img.shape[0])
    src = np.float32([[820,510],[1200,690],[520,510],[320,690]])
    dst = np.float32([[950,0],[950,719],[290,0],[290,719]])
    M = cv2.getPerspectiveTransform(dst,src)
    unwarped = cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
    return unwarped

### Warping images and check the results

In [30]:
image = mpimg.imread('test_images/test4.jpg')
dst = undistort(image)
warped_img = warp(dst)
f, (ax1,ax2) = plt.subplots(1,2,figsize=(20,10))
ax1.set_title("Original")
cv2.line(dst,(820,510),(1200,690),(0,255,0),thickness=2)
cv2.line(dst,(520,510),(320,690),(0,255,0),thickness=2)
cv2.line(warped_img,(950,0),(950,720),(0,255,0),thickness=2)
cv2.line(warped_img,(290,0),(290,720),(0,255,0),thickness=2)
ax1.imshow(dst,cmap='gray')
ax2.set_title("Warped")
ax2.imshow(warped_img,cmap='gray')



### MORE HELPERS

In [31]:
def best_mask(image):
    dst = undistort(image)
    binary_l = hls_select(dst,(100,255),channel = 1)
    binary_s = hls_select(dst,(100,215),channel = 2)
    binary_r = rgb_select(dst,(100,255),channel = 0)
    binary_g = rgb_select(dst,(135,255),channel = 1)
    binary_b = rgb_select(dst,(105,175),channel = 2)
    
    combined_rg = np.zeros_like(binary_r)
    combined_rg[(binary_r == 1) & (binary_g == 1)] = 1
    combined_rgl = np.zeros_like(combined_rg)
    combined_rgl[(combined_rg == 1) & (binary_l == 1)] = 1
    combined_ls = np.zeros_like(binary_l)
    combined_ls[(binary_l == 1) & (binary_s == 1)] = 1
    combined_rb = np.zeros_like(binary_r)
    combined_rb[(binary_r == 1) & (binary_b == 1)] = 1
    
    combined_ls_rb = np.zeros_like(binary_r)
    combined_ls_rb[(combined_ls == 1) | (combined_rb == 1)] = 1
    ksize = 7
    image = dst
    gradx = abs_sobel_thresh(dst, orient='x', sobel_kernel=ksize, thresh=(20, 230))
    dir_binary = dir_threshold(dst, sobel_kernel=ksize, thresh=(0.7, 1.3))
    grady = abs_sobel_thresh(dst, orient='y', sobel_kernel=ksize, thresh=(20, 230))
    combined = np.zeros_like(dir_binary)
    combined[ (dir_binary == 1)] = 1
    combined_sc = np.zeros_like(binary_s)
    combined_sc[(binary_s == 1) | (combined == 1)] = 1
    inv_rgl = np.zeros_like(combined_rgl)
    inv_rgl[(combined_rgl == 0)] = 1
    inv_b = np.zeros_like(combined_rgl)
    inv_b[(binary_b == 0)] = 1
    inv_ls_rb = np.zeros_like(combined_rgl)
    inv_ls_rb[combined_ls_rb == 0] = 1
    final_combined = np.zeros_like(combined_rgl)
    final_combined[((combined_rgl==1) &(combined_sc == 1))]=1
    fin = np.zeros_like(combined_rgl)
    fin[(inv_rgl == 1) & (grady == 1) & (binary_l == 1)] = 1
    ff = np.zeros_like(fin)
    ff[(fin == 1) | (final_combined == 1)] = 1
    test = np.zeros_like(fin)
    test[(inv_b == 1) & (binary_g == 1)] = 1
    xx = np.zeros_like(combined_rgl)
    xx[((test==1) &(combined == 1))]=1
    # de pe binary_g iau linia galbena lejer
    # de pe binary_l iau linia alba semi ljr da trrb inv
    
   
    return xx
def calculate_pos(img,left_fit_cr,right_fit_cr):
    ym_per_pix = 30/720 
    xm_per_pix = 3.7/640
    xMax = img.shape[1]*xm_per_pix
    yMax = img.shape[0]*ym_per_pix
    vehicleCenter = xMax / 2
    lineLeft = left_fit_cr[0]*yMax**2 + left_fit_cr[1]*yMax + left_fit_cr[2]
    lineRight = right_fit_cr[0]*yMax**2 + right_fit_cr[1]*yMax + right_fit_cr[2]
    lineMiddle = lineLeft + (lineRight - lineLeft)/2
    rightDist = vehicleCenter-lineLeft
    leftDist = -(vehicleCenter-lineRight)
    
    diffFromVehicle = lineMiddle - vehicleCenter
    
    return diffFromVehicle,leftDist,rightDist
def diffFromVehicle(diff):
    if diff < 0:
        message = '{:.2f} m right'.format(-diff)
    else:
        message = '{:.2f} m left'.format(diff)
    return message
def add_text(img,left,right,message):
    FONT = cv2.FONT_HERSHEY_DUPLEX
    fontColor = (255,255,255)
    fontScale = 1
    cv2.putText(img, 'Left curvature: {:.0f} m'.format(left), (50, 50), FONT, fontScale, fontColor, 2)
    cv2.putText(img, 'Right curvature: {:.0f} m'.format(right), (50, 100), FONT, fontScale, fontColor, 2)
    cv2.putText(img, 'Vehicle is: {} m of center'.format(message), (50, 150), FONT, fontScale, fontColor, 2)

def measure_curvature_pixels(left_fitx,right_fitx,ploty):
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fitx[0]*y_eval + left_fitx[1])**2)**1.5) / np.absolute(2*left_fitx[0])
    right_curverad = ((1 + (2*right_fitx[0]*y_eval + right_fitx[1])**2)**1.5) / np.absolute(2*right_fitx[0])
    return left_curverad, right_curverad
def measure_curvature_real(left_fitx,right_fitx,ploty):
    
    ym_per_pix = 30/720 
    xm_per_pix = 3.7/640
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fitx[0]*y_eval*ym_per_pix + left_fitx[1])**2)**1.5) / np.absolute(2*left_fitx[0])
    right_curverad = ((1 + (2*right_fitx[0]*y_eval*ym_per_pix + right_fitx[1])**2)**1.5) / np.absolute(2*right_fitx[0])
    return left_curverad, right_curverad
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)


In [32]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    if(len(leftx) == 0 or len(lefty) == 0 or len(rightx) == 0 or len(righty) == 0):
        return 1,1,1,1,1,True
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ym_per_pix = 30/720 
    xm_per_pix = 3.7/640
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty,left_fit_cr,right_fit_cr,False

def search_around_poly(binary_warped,left_fit,right_fit):
   
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    left_fitx, right_fitx, ploty,left_fit_cr,right_fit_cr,valid = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    if valid == True:
        return 1,1,1,1,1,True

    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    

    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    
    
    return result,left_fitx,right_fitx,left_fit_cr,right_fit_cr,False
def find_lane_pixels(binary_warped):
    
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint+100:]) + midpoint + 100
    
    
    nwindows = 9
    
    margin = 100
    
    minpix = 40
    

    window_height = np.int(binary_warped.shape[0]//nwindows)

    nonzero = binary_warped.nonzero()
    
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    leftx_current = leftx_base
    rightx_current = rightx_base

    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):

        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):

    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    if(len(leftx) == 0 or len(lefty) == 0 or len(rightx) == 0 or len(righty) == 0):
        return 1,1,1,1,1,1,1,True

    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ym_per_pix = 30/720 
    xm_per_pix = 3.7/640
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    return out_img,left_fitx,right_fitx,left_fit,right_fit,left_fit_cr,right_fit_cr,False

class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        self.current_fit_cr = [np.array([False])]
        self.current_newfit = np.array([False])
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        self.last_his = 0

In [33]:
def process_img(image):
    global start
    global contor
    global asd
    final_combined = best_mask(image)
    warped_img = warp(final_combined)
    ploty = np.linspace(0, warped_img.shape[0]-1, warped_img.shape[0] )
    warp_zero = np.zeros_like(warped_img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    #print(asd)
    if(contor==1):
        out_img,left_fitx,right_fitx,left_fit,right_fit,left_fit_cr,right_fit_cr,valid = fit_polynomial(warped_img)
        left_line.current_fit = left_fit
        right_line.current_fit = right_fit
        asd = 1
        if valid == True:

            pts_left = np.array([np.transpose(np.vstack([left_line.recent_xfitted, ploty]))])
            pts_right = np.array([np.flipud(np.transpose(np.vstack([right_line.recent_xfitted, ploty])))])
        
            pts = np.hstack((pts_left, pts_right))
            cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
            
            newwarp = unwarp(color_warp)
            
            left,right = measure_curvature_real(left_line.current_fit_cr,right_line.current_fit_cr,ploty)
            message,_,_ = calculate_pos(newwarp,left_line.current_fit_cr,right_line.current_fit_cr)
            result = cv2.addWeighted(undistort(image), 1, newwarp, 0.3, 0)
            add_text(result,left,right,diffFromVehicle(message))
            asd = asd + 1
            return result
        if start == True:
            curvs,curvr = measure_curvature_pixels(left_fitx,right_fitx,ploty)
            #print("Modified radius curvature init")
            left_line.radius_of_curvature = curvs
            right_line.radius_of_curvature = curvr
            left_line.recent_xfitted = left_fitx
            right_line.recent_xfitted = right_fitx
            left_line.current_fit_cr = left_fit_cr
            right_line.current_fit_cr = right_fit_cr
            start = False
        else:
            percent3Upl = left_line.radius_of_curvature * 120/100
            percent3Downl = left_line.radius_of_curvature * 80/100
            percent3Upr = right_line.radius_of_curvature * 120/100
            percent3Downr = right_line.radius_of_curvature * 80/100
            curvs,curvr = measure_curvature_pixels(left_fitx,right_fitx,ploty)
            if(curvs < percent3Upl and curvs > percent3Downl and curvr < percent3Upr and curvr > percent3Downr):
                #print("Modified radius curvature ")
                left_line.radius_of_curvature = curvs
                right_line.radius_of_curvature = curvr
                left_line.recent_xfitted = left_fitx
                right_line.recent_xfitted = right_fitx 
                left_line.current_fit_cr = left_fit_cr
                right_line.current_fit_cr = right_fit_cr
            else:
                pts_left = np.array([np.transpose(np.vstack([left_line.recent_xfitted, ploty]))])
                pts_right = np.array([np.flipud(np.transpose(np.vstack([right_line.recent_xfitted, ploty])))])
                asd = asd + 1
                pts = np.hstack((pts_left, pts_right))
                cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
                
                newwarp = unwarp(color_warp)
                left,right = measure_curvature_real(left_line.current_fit_cr,right_line.current_fit_cr,ploty)
                message,_,_ = calculate_pos(newwarp,left_fit_cr,right_fit_cr)
                result = cv2.addWeighted(undistort(image), 1, newwarp, 0.3, 0)
                add_text(result,left,right,diffFromVehicle(message))
                
            #contor = 1
                return result
            
        
        
        
    if(contor > 1 ):

        result,left_fitx,right_fitx,left_fit_cr,right_fit_cr,valid = search_around_poly(warped_img,left_line.current_fit,right_line.current_fit )
        
        if valid == True:
            
            
            pts_left = np.array([np.transpose(np.vstack([left_line.recent_xfitted, ploty]))])
            pts_right = np.array([np.flipud(np.transpose(np.vstack([right_line.recent_xfitted, ploty])))])
        
            pts = np.hstack((pts_left, pts_right))
            cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
            
            newwarp = unwarp(color_warp)
            left,right = measure_curvature_real(left_line.current_fit_cr,right_line.current_fit_cr,ploty)
            message,_,_ = calculate_pos(newwarp,left_line.current_fit_cr,right_line.current_fit_cr)
            result = cv2.addWeighted(undistort(image), 1, newwarp, 0.3, 0)
            add_text(result,left,right,diffFromVehicle(message))
            asd = asd + 1
            return result
    if contor > 1 :
        curvs,curvr = measure_curvature_pixels(left_fitx,right_fitx,ploty)
        percent3Upl = left_line.radius_of_curvature * 104/100
        percent3Downl = left_line.radius_of_curvature * 96/100
        percent3Upr = right_line.radius_of_curvature * 104/100
        percent3Downr = right_line.radius_of_curvature * 96/100
        if(curvs > percent3Upl or curvs < percent3Downl or curvr > percent3Upr or curvr < percent3Downr):
            asd = asd + 1
            pts_left = np.array([np.transpose(np.vstack([left_line.recent_xfitted, ploty]))])
            pts_right = np.array([np.flipud(np.transpose(np.vstack([right_line.recent_xfitted, ploty])))])

            pts = np.hstack((pts_left, pts_right))
            cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

            newwarp = unwarp(color_warp)
            left,right = measure_curvature_real(left_fit_cr,right_fit_cr,ploty)
            message,_,_ = calculate_pos(newwarp,left_fit_cr,right_fit_cr)
            result = cv2.addWeighted(undistort(image), 1, newwarp, 0.3, 0)
            add_text(result,left,right,diffFromVehicle(message))
            #contor = 1
            if asd > 4 : 
                contor = 1

            return result
    
       
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    newwarp = unwarp(color_warp)
    left,right = measure_curvature_real(left_fit_cr,right_fit_cr,ploty)
    message,_,_ = calculate_pos(newwarp,left_fit_cr,right_fit_cr)
    result = cv2.addWeighted(undistort(image), 1, newwarp, 0.3, 0)
    add_text(result,left,right,diffFromVehicle(message))
    

    contor = contor + 1
    return result

In [34]:
start = True
contor = 1
asd = 1
left_line = Line()
right_line = Line()
white_output = 'video/project_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_img) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   0%|                                                                            | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video video/project_video.mp4.
Moviepy - Writing video video/project_video.mp4



Moviepy - Done !
Moviepy - video ready video/project_video.mp4
Wall time: 5min 37s
